In [ ]:
# API de reconnaissance d'images avec Flask et TensorFlow
# Notebook 

# 1. Installation des bibliothèques nécessaires
!pip install flask flask-cors pillow tensorflow

# 2. Importation des bibliothèques
import tensorflow as tf
from tensorflow.keras import models
import numpy as np
from PIL import Image
import io
import os
import base64
from flask import Flask, request, jsonify
from flask_cors import CORS
import matplotlib.pyplot as plt
from google.colab.output import eval_js
from IPython.display import display, Javascript, HTML

# 3. Chargement du modèle (option 1 - utiliser le modèle que vous avez déjà entraîné)
# Décommentez cette section si vous avez déjà entraîné et sauvegardé un modèle
"""
try:
    model = models.load_model('fashion_mnist_model.h5')
    print("Modèle chargé avec succès!")
except:
    print("Erreur: Impossible de charger le modèle entraîné.")
"""

# 4. Chargement du modèle (option 2 - utiliser un modèle pré-entraîné)
# Cette option est plus rapide si vous n'avez pas encore entraîné de modèle

# Charger le jeu de données Fashion MNIST
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

# Prétraiter les données
train_images = train_images / 255.0
test_images = test_images / 255.0

# Créer et entraîner rapidement un modèle simple
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=32, verbose=1)

# Évaluer le modèle
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Précision sur données de test: {test_acc:.4f}")

# Définir les noms des classes
class_names = ['T-shirt/haut', 'Pantalon', 'Pull', 'Robe', 'Manteau',
               'Sandale', 'Chemise', 'Baskets', 'Sac', 'Bottine']

# 5. Création de l'application Flask
app = Flask(__name__)
CORS(app)  # Permet l'accès depuis d'autres domaines (important pour les applications web)

# Fonction de prétraitement des images
def preprocess_image(image):
    """
    Prétraite une image pour qu'elle soit compatible avec notre modèle
    
    Args:
        image: Image PIL
    
    Returns:
        np.array: Image prétraitée et formatée pour le modèle
    """
    # Convertir en niveaux de gris si nécessaire
    if image.mode != 'L':
        image = image.convert('L')
    
    # Redimensionner à 28x28 pixels
    image = image.resize((28, 28))
    
    # Convertir en array numpy
    img_array = np.array(image)
    
    # Normaliser les valeurs de pixels à [0,1]
    img_array = img_array / 255.0
    
    # Ajouter une dimension de batch
    img_array = img_array.reshape(1, 28, 28)
    
    return img_array

# Route principale
@app.route('/')
def home():
    return """
    <h1>API de reconnaissance d'images</h1>
    <p>Utilisez le point d'accès /predict pour classifier des images de vêtements.</p>
    """

# Route pour les prédictions
@app.route('/predict', methods=['POST'])
def predict():
    # Vérifier si la requête contient une image
    if 'image' not in request.files:
        return jsonify({'error': 'Aucune image trouvée dans la requête'}), 400
    
    # Récupérer l'image
    file = request.files['image']
    
    try:
        # Lire l'image
        img = Image.open(file.stream)
        
        # Prétraiter l'image
        processed_image = preprocess_image(img)
        
        # Faire la prédiction
        predictions = model.predict(processed_image)
        
        # Récupérer la classe avec la plus haute probabilité
        predicted_class = np.argmax(predictions[0])
        confidence = float(predictions[0][predicted_class])
        
        # Préparer la réponse
        response = {
            'prediction': {
                'class_id': int(predicted_class),
                'class_name': class_names[predicted_class],
                'confidence': float(confidence)
            },
            'all_probabilities': {
                class_names[i]: float(predictions[0][i]) for i in range(len(class_names))
            }
        }
        
        return jsonify(response)
    
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# 6. Route pour obtenir des informations sur le modèle
@app.route('/info', methods=['GET'])
def model_info():
    model_summary = []
    
    # Capturer le résumé du modèle dans une liste
    model.summary(print_fn=lambda x: model_summary.append(x))
    
    return jsonify({
        'model_name': 'Fashion MNIST Classifier',
        'input_shape': model.input_shape[1:],
        'output_shape': model.output_shape[1:],
        'number_of_classes': len(class_names),
        'classes': class_names,
        'model_summary': model_summary
    })

# 7. Configuration pour exécuter Flask dans Colab
import threading
from google.colab.output import eval_js
from IPython.display import display, Javascript

# Fonction pour exécuter Flask dans Colab
def run_flask(app):
    from google.colab.output import eval_js
    from IPython.display import display, Javascript
    import threading
    
    def b64_to_pil(b64_str):
        import base64
        import io
        from PIL import Image
        img_data = base64.b64decode(b64_str)
        return Image.open(io.BytesIO(img_data))
    
    # Démarrer le serveur Flask dans un thread séparé
    threading.Thread(target=lambda: app.run(host='0.0.0.0', port=8000, debug=False, use_reloader=False)).start()
    
    # Afficher un message pour indiquer que le serveur est en cours d'exécution
    display(HTML("""
    <div style="background-color: #4CAF50; color: white; padding: 12px; margin: 10px 0; border-radius: 4px;">
        <h3 style="margin: 0;">API Flask en cours d'exécution!</h3>
        <p style="margin: 5px 0 0 0;">Utilisez l'interface ci-dessous pour tester votre API.</p>
    </div>
    """))
    
    # Interface pour télécharger et tester l'API
    display(HTML("""
    <div style="width: 100%; max-width: 800px; margin: 0 auto; padding: 20px; border: 1px solid #ddd; border-radius: 8px;">
        <h2>Testez votre API de reconnaissance d'images</h2>
        <div style="margin-bottom: 20px;">
            <label for="file-upload" style="display: block; margin-bottom: 10px;">Sélectionnez une image:</label>
            <input type="file" id="file-upload" accept="image/*" style="display: block; margin-bottom: 10px;">
            <button id="predict-button" style="background-color: #4CAF50; color: white; padding: 10px 15px; border: none; border-radius: 4px; cursor: pointer;">Prédire</button>
        </div>
        <div id="spinner" style="display: none; text-align: center; margin: 20px 0;">
            <div style="border: 4px solid #f3f3f3; border-top: 4px solid #3498db; border-radius: 50%; width: 30px; height: 30px; animation: spin 2s linear infinite; margin: 0 auto;"></div>
            <p>Traitement en cours...</p>
            <style>@keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } }</style>
        </div>
        <div id="result" style="margin-top: 20px; display: none;">
            <h3>Résultat:</h3>
            <div id="prediction-result" style="padding: 15px; background-color: #f9f9f9; border-radius: 4px;"></div>
        </div>
        <div id="error" style="margin-top: 20px; display: none; color: red;"></div>
        <div id="image-preview" style="margin-top: 20px; text-align: center;"></div>
    </div>

    <script>
        document.getElementById('predict-button').addEventListener('click', function() {
            const fileInput = document.getElementById('file-upload');
            const file = fileInput.files[0];
            
            if (!file) {
                document.getElementById('error').textContent = 'Veuillez sélectionner une image';
                document.getElementById('error').style.display = 'block';
                document.getElementById('result').style.display = 'none';
                return;
            }
            
            const spinner = document.getElementById('spinner');
            const result = document.getElementById('result');
            const error = document.getElementById('error');
            
            spinner.style.display = 'block';
            result.style.display = 'none';
            error.style.display = 'none';
            
            // Afficher l'aperçu de l'image
            const reader = new FileReader();
            reader.onload = function(e) {
                document.getElementById('image-preview').innerHTML = `
                    <img src="${e.target.result}" style="max-width: 300px; max-height: 300px; border: 1px solid #ddd; border-radius: 4px;">
                `;
            };
            reader.readAsDataURL(file);
            
            // Envoyer l'image à l'API
            const formData = new FormData();
            formData.append('image', file);
            
            fetch('http://localhost:8000/predict', {
                method: 'POST',
                body: formData
            })
            .then(response => {
                if (!response.ok) {
                    throw new Error('Erreur réseau ou erreur serveur');
                }
                return response.json();
            })
            .then(data => {
                spinner.style.display = 'none';
                result.style.display = 'block';
                
                // Afficher les résultats
                const predictionResult = document.getElementById('prediction-result');
                
                // Créer un tableau de classification avec barre de confiance
                let resultsHTML = `
                    <div style="margin-bottom: 15px;">
                        <h4 style="margin: 0 0 10px 0;">Prédiction: ${data.prediction.class_name} (${(data.prediction.confidence * 100).toFixed(2)}%)</h4>
                    </div>
                    <table style="width: 100%; border-collapse: collapse;">
                        <tr>
                            <th style="text-align: left; padding: 8px; border-bottom: 1px solid #ddd;">Classe</th>
                            <th style="text-align: left; padding: 8px; border-bottom: 1px solid #ddd;">Confiance</th>
                            <th style="text-align: left; padding: 8px; border-bottom: 1px solid #ddd;"></th>
                        </tr>
                `;
                
                // Trier les probabilités par ordre décroissant
                const sortedProbabilities = Object.entries(data.all_probabilities)
                    .sort((a, b) => b[1] - a[1]);
                
                sortedProbabilities.forEach(([className, prob]) => {
                    const percentage = (prob * 100).toFixed(2);
                    const isHighest = className === data.prediction.class_name;
                    
                    resultsHTML += `
                        <tr>
                            <td style="padding: 8px; border-bottom: 1px solid #ddd; ${isHighest ? 'font-weight: bold;' : ''}">
                                ${className}
                            </td>
                            <td style="padding: 8px; border-bottom: 1px solid #ddd; ${isHighest ? 'font-weight: bold;' : ''}">
                                ${percentage}%
                            </td>
                            <td style="padding: 8px; border-bottom: 1px solid #ddd;">
                                <div style="background-color: #e0e0e0; border-radius: 4px; height: 20px; width: 100%;">
                                    <div style="background-color: ${isHighest ? '#4CAF50' : '#3498db'}; height: 20px; border-radius: 4px; width: ${percentage}%"></div>
                                </div>
                            </td>
                        </tr>
                    `;
                });
                
                resultsHTML += `</table>`;
                predictionResult.innerHTML = resultsHTML;
            })
            .catch(error => {
                spinner.style.display = 'none';
                document.getElementById('error').textContent = 'Erreur: ' + error.message;
                document.getElementById('error').style.display = 'block';
                console.error('Erreur:', error);
            });
        });
    </script>
    """))

# 8. Lancer l'application Flask
run_flask(app)

# 9. Exercices d'amélioration (à faire par les étudiants)
"""
Exercice 1: Ajouter une fonction de cache pour les prédictions
----------------------------------------------------------------
Pour éviter de faire des prédictions redondantes sur des images similaires,
ajoutez un mécanisme de cache simple.

Conseil: Vous pouvez utiliser un dictionnaire où les clés sont des hash
des images et les valeurs sont les résultats de prédiction.

Exercice 2: Ajouter une limite de requêtes
----------------------------------------------------------------
Implémentez une limite de requêtes par IP pour éviter une surcharge du service.

Conseil: Vous pouvez utiliser un dictionnaire pour stocker le nombre
de requêtes par IP et le timestamp de la dernière requête.

Exercice 3: Améliorer le prétraitement des images
----------------------------------------------------------------
Modifiez la fonction de prétraitement pour qu'elle puisse gérer
différents formats et tailles d'images d'entrée.

Conseil: Ajoutez la détection automatique de fond et le recadrage
pour ne garder que l'objet principal.
"""

# 10. Pour aller plus loin: Déploiement de l'API
"""
Dans un environnement de production, vous voudriez déployer cette API
sur un serveur web. Voici les étapes que vous pourriez suivre:

1. Sauvegarder le modèle:
model.save('mon_modele_fashion.h5')

2. Créer un fichier app.py contenant le code Flask

3. Créer un fichier requirements.txt avec les dépendances:
flask==2.0.1
flask-cors==3.0.10
tensorflow==2.8.0
pillow==8.3.1
numpy==1.21.2
gunicorn==20.1.0

4. Déployer sur une plateforme comme Heroku, Google Cloud Run, ou AWS Elastic Beanstalk

5. Pour un déploiement local ou sur votre propre serveur:
   - Installer les dépendances: pip install -r requirements.txt
   - Lancer le serveur: gunicorn -w 4 -b 0.0.0.0:8000 app:app
"""

# Ce notebook a été conçu pour être utilisé dans Google Colab.
# Si vous souhaitez l'utiliser en local, vous devrez ajuster la partie d'exécution de Flask.